### Single-Agent System

In [ ]:
import subprocess
import psutil
import random
import numpy as np
from deap import base, creator, tools, algorithms
from scipy.stats import norm
from scipy.linalg import solve_discrete_are, inv
import matplotlib.pyplot as plt

# Problem parameters
A = np.array([[1, 0.1], [0, 1]])  # State transition matrix
B = np.array([[0.005], [1]])        # Control matrix
N = 100  # Number of time steps
M = 100  # Number of disturbance sequences
theta = 0.05  # Quantile parameter
threshold = 1.12  # User-defined threshold for ||K * e[t]||_2

# Generate disturbance sequences w^j(t)
#np.random.seed(0)
dist1 = norm(loc=-0.01, scale=np.sqrt(0.005))  # N(-0.05, 0.01)
#dist2 = norm(loc=0.13, scale=np.sqrt(0.1))    # N(0.03, 0.1)
samples_1 = dist1.rvs(size=(M, N))  # Samples for the first element
#samples_2 = dist2.rvs(size=(M, N))  # Samples for the second element
shape = 5.5  # shape parameter (k)
theta = 0.005  # scale parameter (theta)
# Generate Gamma-distributed samples
gamma_samples = np.random.gamma(shape=shape, scale=theta, size=(M, N))
# Flip half of the samples to negative values to make it symmetric
sym_gamma_samples = gamma_samples * np.random.choice([-1, 1], size=(M, N))
# Use the symmetric Gamma samples for samples_2
samples_2 = sym_gamma_samples

w = np.stack([samples_1, samples_2], axis=-1)

plt.figure(figsize=(8, 8))
plt.scatter(samples_1, samples_2, c='blue', alpha=0.5, edgecolors='k')
plt.xlabel('Samples 1')
plt.ylabel('Samples 2')
plt.title('2D Scatter Plot of Samples 1 vs Samples 2')
plt.grid(True)
plt.show()

# Discrete-time LQR initialization
Q = np.eye(2)  # State cost matrix
R = np.eye(1)  # Control cost matrix

# Solve the discrete-time algebraic Riccati equation (DARE)
P = solve_discrete_are(A, B, Q, R)

# Compute the LQR gain
K = -np.dot(inv(R + B.T @ P @ B), B.T @ P @ A)
print('Discrete-time LQR Gain Matrix K:', K)

# State dynamics
def state_dynamics(K, e, w):
    A_BK = A + B @ K
    e_next = A_BK @ e + w
    return e_next

# Compute R^j for a given K
def compute_R_j(K, j):
    e = np.zeros((N+1, A.shape[0]))
    for t in range(N):
        e[t+1] = state_dynamics(K, e[t], w[j, t])
    return np.max(np.linalg.norm(e[1:], axis=1))

# Compute Ru^j for a given K
def compute_Ru_j(K, j):
    e = np.zeros((N+1, A.shape[0]))
    Ke = np.zeros((N, 1))
    for t in range(N):
        e[t+1] = state_dynamics(K, e[t], w[j, t])
        Ke[t] = K @ e[t]
    return np.max(np.linalg.norm(Ke[1:], axis=1))

# Objective function to be minimized
def objective_function(K, theta, M, threshold):
    K = np.array(K).reshape(B.shape[1], A.shape[0])
    R_values = [compute_R_j(K, j) for j in range(M)]
    Ru_values = [compute_Ru_j(K, j) for j in range(M)]
    quantile_value = np.quantile(R_values, 1 - theta)
    quantile_value_for_u = np.quantile(Ru_values, 1 - theta)
    
    # Penalize if the constraint is violated
    penalty = 0
    for j in range(M):
        for t in range(N):
            if quantile_value_for_u > threshold:
                penalty += quantile_value_for_u - threshold
    
    return quantile_value + 1.0 * quantile_value_for_u + penalty,

# Genetic Algorithm Setup
def genetic_algorithm_solver(theta, M, u_min, u_max, threshold, population_size=100, generations=50, cxpb=0.7, mutpb=0.2):
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, u_min, u_max)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, B.shape[1] * A.shape[0])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("evaluate", objective_function, theta=theta, M=M, threshold=threshold)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=population_size)
    
    # Stats for tracking
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Running the algorithm
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=generations, stats=stats, verbose=True)
    
    return population, logbook

# Parameters for the GA
u_min = -10  # Lower bound of each element of state-feedback gain K
u_max = 10   # Upper bound of each element of state-feedback gain K
population_size = 150  # Number of candidates at each generation
generations = 50       # Number of iterations

# Running the solver
population, logbook = genetic_algorithm_solver(theta, M, u_min, u_max, threshold, population_size, generations)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_K = np.array(best_individual).reshape(B.shape[1], A.shape[0])
print('Best individual (K):', best_K)
print('Best fitness:', best_individual.fitness.values[0])

process = psutil.Process()

# Get memory info
memory_info = process.memory_info()
print(f"RSS (Resident Set Size): {memory_info.rss / (1024 ** 2):.2f} MB")  # Resident memory
print(f"VMS (Virtual Memory Size): {memory_info.vms / (1024 ** 2):.2f} MB")  # Virtual memory


## Multi-Agent System

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from scipy.stats import norm
from scipy.linalg import solve_discrete_are, inv
import matplotlib.pyplot as plt

# Problem parameters
I6 = np.eye(6)
A = np.kron(I6, np.array([[1, 0.1], [0, 1]]))  # State transition matrix
B = np.kron(I6, np.array([[0.005], [1]]))      # Control matrix
N = 100  # Number of time steps
M = 100  # Number of disturbance sequences
theta = 0.05  # Quantile parameter
threshold = 1.12  # User-defined threshold for ||K * e[t]||_2

# Generate disturbance sequences w^j(t)
def disturbance_samples(mean, var, shape, theta, M, N):
    dist1 = norm(loc=mean, scale=np.sqrt(var))  # Normal distribution
    samples_1 = dist1.rvs(size=(M, N))
    samples_3 = dist1.rvs(size=(M, N))
    samples_5 = dist1.rvs(size=(M, N))
    samples_7 = dist1.rvs(size=(M, N))
    samples_9 = dist1.rvs(size=(M, N))
    samples_11 = dist1.rvs(size=(M, N))

    gamma_samples2 = np.random.gamma(shape=shape, scale=theta, size=(M, N))
    sym_gamma_samples2 = gamma_samples2 * np.random.choice([-1, 1], size=(M, N))
    samples_2 = sym_gamma_samples2

    gamma_samples4 = np.random.gamma(shape=shape, scale=theta, size=(M, N))
    sym_gamma_samples4 = gamma_samples4 * np.random.choice([-1, 1], size=(M, N))
    samples_4 = sym_gamma_samples4

    gamma_samples6 = np.random.gamma(shape=shape, scale=theta, size=(M, N))
    sym_gamma_samples6 = gamma_samples6 * np.random.choice([-1, 1], size=(M, N))
    samples_6 = sym_gamma_samples6

    gamma_samples8 = np.random.gamma(shape=shape, scale=theta, size=(M, N))
    sym_gamma_samples8 = gamma_samples8 * np.random.choice([-1, 1], size=(M, N))
    samples_8 = sym_gamma_samples8

    gamma_samples10 = np.random.gamma(shape=shape, scale=theta, size=(M, N))
    sym_gamma_samples10 = gamma_samples10 * np.random.choice([-1, 1], size=(M, N))
    samples_10 = sym_gamma_samples10

    gamma_samples12 = np.random.gamma(shape=shape, scale=theta, size=(M, N))
    sym_gamma_samples12 = gamma_samples12 * np.random.choice([-1, 1], size=(M, N))
    samples_12 = sym_gamma_samples12

    # Stack all samples along the last axis
    return np.stack([samples_1, samples_2, samples_3, samples_4, samples_5, samples_6,
                     samples_7, samples_8, samples_9, samples_10, samples_11, samples_12], axis=-1)

# Parameters for calling disturbance_samples
mean = -0.01
var = 0.005
shape = 5.5
theta_g = 0.005

# Generate disturbances
w = disturbance_samples(mean, var, shape, theta_g, M, N)  # Shape: (M, N, 12)

# Discrete-time LQR initialization
Q = np.eye(12)  # State cost matrix
R = np.eye(6)   # Control cost matrix

# Solve the discrete-time algebraic Riccati equation (DARE)
P = solve_discrete_are(A, B, Q, R)

# Compute the LQR gain
K = -np.dot(inv(R + B.T @ P @ B), B.T @ P @ A)
print('Discrete-time LQR Gain Matrix K:', K)

# State dynamics
def state_dynamics(K, e, w_t):
    A_BK = A + B @ K
    e_next = A_BK @ e + w_t
    return e_next

# Compute R^j for a given K
def compute_R_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))  # Shape: (N+1, 12)
    for t in range(N):
        e[t + 1] = state_dynamics(K, e[t], w[j, t])
    return np.max(np.linalg.norm(e[1:], axis=1))

# Compute Ru^j for a given K
def compute_Ru_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))   # Shape: (N+1, 12)
    Ke = np.zeros((N, K.shape[0]))      # Corrected shape: (N, 6)
    for t in range(N):
        e[t + 1] = state_dynamics(K, e[t], w[j, t])
        Ke[t] = K @ e[t]                # Ke[t] has shape (6,)
    return np.max(np.linalg.norm(Ke[1:], axis=1))

# Objective function to be minimized
def objective_function(K_flat, theta, M, threshold):
    K = np.array(K_flat).reshape(B.shape[1], A.shape[0])  # Shape: (6, 12)
    R_values = [compute_R_j(K, j) for j in range(M)]
    Ru_values = [compute_Ru_j(K, j) for j in range(M)]
    quantile_value = np.quantile(R_values, 1 - theta)
    quantile_value_for_u = np.quantile(Ru_values, 1 - theta)
    
    # Penalize if the constraint is violated
    penalty = 0
    if quantile_value_for_u > threshold:
        penalty += (quantile_value_for_u - threshold) * M * N  # Adjusted penalty calculation

    return quantile_value + quantile_value_for_u + penalty,

# Genetic Algorithm Setup
def genetic_algorithm_solver(theta, M, u_min, u_max, threshold, population_size=100, generations=50, cxpb=0.7, mutpb=0.2):
    # Ensure the classes are not created multiple times
    if not hasattr(creator, 'FitnessMin'):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, 'Individual'):
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, u_min, u_max)
    num_genes = B.shape[1] * A.shape[0]  # Total number of genes in the individual
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, num_genes)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("evaluate", objective_function, theta=theta, M=M, threshold=threshold)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=population_size)
    
    # Stats for tracking
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: np.mean([v[0] for v in x]))
    stats.register("std", lambda x: np.std([v[0] for v in x]))
    stats.register("min", lambda x: np.min([v[0] for v in x]))
    stats.register("max", lambda x: np.max([v[0] for v in x]))

    # Running the algorithm
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb,
                                              ngen=generations, stats=stats, verbose=True)
    
    return population, logbook

# Parameters for the GA
u_min = -10  # Lower bound of each element of state-feedback gain K
u_max = 10   # Upper bound of each element of state-feedback gain K
population_size = 150  # Number of candidates at each generation
generations = 50       # Number of iterations

# Running the solver
population, logbook = genetic_algorithm_solver(theta, M, u_min, u_max, threshold, population_size, generations)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_K = np.array(best_individual).reshape(B.shape[1], A.shape[0])
print('Best individual (K):', best_K)
print('Best fitness:', best_individual.fitness.values[0])


### Improved GA for final submission (double integrator setup)

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from scipy.stats import norm
from scipy.linalg import solve_discrete_are, inv
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

# Problem parameters
A = np.array([[1, 0.5], [0, 1]])  # dynamics matrix 
B = np.array([[0], [0.5]])      # input matrix
N = 100  # Number of time steps
M = 1000  # Number of disturbance sequences
theta = 0.05  # Quantile parameter
threshold = 1.12  # User-defined threshold for ||K * e[t]||_2

# Parameters for the GA
u_min = -10  # Lower bound of each element of state-feedback gain K
u_max = 10   # Upper bound of each element of state-feedback gain K
population_size = 150  # Increased population size
generations = 50      # Increased number of generations
cxpb = 0.8             # Crossover probability
mutpb = 0.1            # Mutation probability

# Generate disturbance sequences w^j(t)
def disturbance_samples(mean, var, shape, theta, M, N):
    dist1 = norm(loc=mean, scale=np.sqrt(var))  # Normal distribution
    samples_normal = [dist1.rvs(size=(M, N)) for _ in range(1)]

    gamma_samples = [np.random.gamma(shape=shape, scale=theta, size=(M, N)) * np.random.choice([-1, 1], size=(M, N)) for _ in range(1)]

    # Stack all samples along the last axis
    return np.stack(samples_normal + gamma_samples, axis=-1)  # Shape: (M, N, 12)

# Parameters for calling disturbance_samples
mean = -0.01
var = 0.005
shape = 5.5
theta_g = 0.005

# Generate disturbances
w = disturbance_samples(mean, var, shape, theta_g, M, N)  # Shape: (M, N, 12)

plt.figure(figsize=(8, 8))
for i in range(M):
    plt.scatter(w[i, :, 0], w[i, :, 1], label=f'Sequence {i+1}', alpha=0.6)

plt.title('Gamma vs Normal Disturbance')
plt.xlabel('Gamma Disturbance (w[:, :, 0])')
plt.ylabel('Normal Disturbance (w[:, :, 1])')
plt.grid(True)
plt.show()

# Discrete-time LQR initialization
Q = np.eye(2)  # State cost matrix
R = np.eye(1)   # Control cost matrix

# Solve the discrete-time algebraic Riccati equation (DARE)
P = solve_discrete_are(A, B, Q, R)

# Compute the LQR gain
K_LQR = -np.dot(inv(R + B.T @ P @ B), B.T @ P @ A)
print('Discrete-time LQR Gain Matrix K_LQR:', K_LQR)

# State dynamics
def state_dynamics(K, e, w_t):
    A_BK = A + B @ K
    e_next = A_BK @ e + w_t
    return e_next

# Compute R^j for a given K
def compute_R_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))  # Shape: (N+1, 12)
    A_BK = A + B @ K
    w_j = w[j]  # Shape: (N, 12)
    for t in range(N):
        e[t + 1] = A_BK @ e[t] + w_j[t]
    return np.max(np.linalg.norm(e[1:], axis=1))

# Compute Ru^j for a given K
def compute_Ru_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))   # Shape: (N+1, 12)
    Ke = np.zeros((N, K.shape[0]))      # Shape: (N, 6)
    A_BK = A + B @ K
    w_j = w[j]  # Shape: (N, 12)
    for t in range(N):
        e[t + 1] = A_BK @ e[t] + w_j[t]
        Ke[t] = K @ e[t]
    return np.max(np.linalg.norm(Ke[1:], axis=1))

# Objective function to be minimized
def objective_function(K_flat, theta, M, threshold):
    K = np.array(K_flat).reshape(B.shape[1], A.shape[0])  # Shape: (6, 12)
    R_values = [compute_R_j(K, j) for j in range(M)]
    Ru_values = [compute_Ru_j(K, j) for j in range(M)]
    quantile_value = np.quantile(R_values, 1 - theta)
    quantile_value_for_u = np.quantile(Ru_values, 1 - theta)

    # Penalize if the constraint is violated
    penalty = 0
    if quantile_value_for_u > threshold:
        penalty += 1000 * (quantile_value_for_u - threshold)  # Adjusted penalty scaling

    return quantile_value + 0.5 * quantile_value_for_u + penalty,

# Custom population initialization
def initialize_population(toolbox, population_size, K_initial, init_ratio=0.1):
    num_initialized = int(population_size * init_ratio)
    population = []

    K_initial_flat = K_initial.flatten()

    # Create individuals initialized with K_initial
    for _ in range(num_initialized):
        individual = creator.Individual(K_initial_flat.tolist())
        population.append(individual)

    # Create the rest of the population randomly
    num_random = population_size - num_initialized
    for _ in range(num_random):
        individual = toolbox.individual()
        population.append(individual)

    return population

# Genetic Algorithm Setup
def genetic_algorithm_solver(theta, M, u_min, u_max, threshold, K_initial, population_size=100, generations=50, cxpb=0.7, mutpb=0.2):
    # Ensure the classes are not created multiple times
    if not hasattr(creator, 'FitnessMin'):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, 'Individual'):
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, u_min, u_max)
    num_genes = B.shape[1] * A.shape[0]  # Total number of genes in the individual
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, num_genes)
    toolbox.register("population", initialize_population, toolbox=toolbox, population_size=population_size, K_initial=K_initial, init_ratio=0.1)

    toolbox.register("evaluate", objective_function, theta=theta, M=M, threshold=threshold)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()

    # Stats for tracking
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Running the algorithm
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb,
                                              ngen=generations, stats=stats, verbose=True)

    return population, logbook

# Running the solver
population, logbook = genetic_algorithm_solver(theta, M, u_min, u_max, threshold, K_LQR, population_size, generations, cxpb, mutpb)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_K = np.array(best_individual).reshape(B.shape[1], A.shape[0])
print('Best individual (K):', best_K)
print('Best fitness:', best_individual.fitness.values[0])

# Compare with LQR performance
print('LQR Gain Matrix K_LQR:', K_LQR)

# Evaluate LQR Controller
lqr_fitness = objective_function(K_LQR.flatten(), theta, M, threshold)[0]
print('LQR Controller Fitness:', lqr_fitness)

# Evaluate Best GA Controller
ga_fitness = objective_function(best_K.flatten(), theta, M, threshold)[0]
print('Best GA Controller Fitness:', ga_fitness)


In [ ]:
w.shape


## Improved GA-based solution with LQR initialization (Multi-Agent System)

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from scipy.stats import norm
from scipy.linalg import solve_discrete_are, inv
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

# Problem parameters
I6 = np.eye(6)
A = np.kron(I6, np.array([[1, 0.1], [0, 1]]))  # State transition matrix (12x12)
B = np.kron(I6, np.array([[0.005], [1]]))      # Control matrix (12x6)
N = 100  # Number of time steps
M = 100  # Number of disturbance sequences
theta = 0.05  # Quantile parameter
threshold = 1.12  # User-defined threshold for ||K * e[t]||_2

# Generate disturbance sequences w^j(t)
def disturbance_samples(mean, var, shape, theta, M, N):
    dist1 = norm(loc=mean, scale=np.sqrt(var))  # Normal distribution
    samples_normal = [dist1.rvs(size=(M, N)) for _ in range(6)]

    gamma_samples = [np.random.gamma(shape=shape, scale=theta, size=(M, N)) * np.random.choice([-1, 1], size=(M, N)) for _ in range(6)]

    # Stack all samples along the last axis
    return np.stack(samples_normal + gamma_samples, axis=-1)  # Shape: (M, N, 12)

# Parameters for calling disturbance_samples
mean = -0.01
var = 0.005
shape = 5.5
theta_g = 0.005

# Generate disturbances
w = disturbance_samples(mean, var, shape, theta_g, M, N)  # Shape: (M, N, 12)

# Discrete-time LQR initialization
Q = np.eye(12)  # State cost matrix
R = np.eye(6)   # Control cost matrix

# Solve the discrete-time algebraic Riccati equation (DARE)
P = solve_discrete_are(A, B, Q, R)

# Compute the LQR gain
K_LQR = -np.dot(inv(R + B.T @ P @ B), B.T @ P @ A)
print('Discrete-time LQR Gain Matrix K_LQR:', K_LQR)

# State dynamics
def state_dynamics(K, e, w_t):
    A_BK = A + B @ K
    e_next = A_BK @ e + w_t
    return e_next

# Compute R^j for a given K
def compute_R_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))  # Shape: (N+1, 12)
    A_BK = A + B @ K
    w_j = w[j]  # Shape: (N, 12)
    for t in range(N):
        e[t + 1] = A_BK @ e[t] + w_j[t]
    return np.max(np.linalg.norm(e[1:], axis=1))

# Compute Ru^j for a given K
def compute_Ru_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))   # Shape: (N+1, 12)
    Ke = np.zeros((N, K.shape[0]))      # Shape: (N, 6)
    A_BK = A + B @ K
    w_j = w[j]  # Shape: (N, 12)
    for t in range(N):
        e[t + 1] = A_BK @ e[t] + w_j[t]
        Ke[t] = K @ e[t]
    return np.max(np.linalg.norm(Ke[1:], axis=1))

# Objective function to be minimized
def objective_function(K_flat, theta, M, threshold):
    K = np.array(K_flat).reshape(B.shape[1], A.shape[0])  # Shape: (6, 12)
    R_values = [compute_R_j(K, j) for j in range(M)]
    Ru_values = [compute_Ru_j(K, j) for j in range(M)]
    quantile_value = np.quantile(R_values, 1 - theta)
    quantile_value_for_u = np.quantile(Ru_values, 1 - theta)

    # Penalize if the constraint is violated
    penalty = 0
    if quantile_value_for_u > threshold:
        penalty += 1000 * (quantile_value_for_u - threshold)  # Adjusted penalty scaling

    return quantile_value + 0.5 * quantile_value_for_u + penalty,

# Custom population initialization
def initialize_population(toolbox, population_size, K_initial, init_ratio=0.1):
    num_initialized = int(population_size * init_ratio)
    population = []

    K_initial_flat = K_initial.flatten()

    # Create individuals initialized with K_initial
    for _ in range(num_initialized):
        individual = creator.Individual(K_initial_flat.tolist())
        population.append(individual)

    # Create the rest of the population randomly
    num_random = population_size - num_initialized
    for _ in range(num_random):
        individual = toolbox.individual()
        population.append(individual)

    return population

# Genetic Algorithm Setup
def genetic_algorithm_solver(theta, M, u_min, u_max, threshold, K_initial, population_size=100, generations=50, cxpb=0.7, mutpb=0.2):
    # Ensure the classes are not created multiple times
    if not hasattr(creator, 'FitnessMin'):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, 'Individual'):
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, u_min, u_max)
    num_genes = B.shape[1] * A.shape[0]  # Total number of genes in the individual
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, num_genes)
    toolbox.register("population", initialize_population, toolbox=toolbox, population_size=population_size, K_initial=K_initial, init_ratio=0.1)

    toolbox.register("evaluate", objective_function, theta=theta, M=M, threshold=threshold)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()

    # Stats for tracking
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Running the algorithm
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb,
                                              ngen=generations, stats=stats, verbose=True)

    return population, logbook

# Parameters for the GA
u_min = -10  # Lower bound of each element of state-feedback gain K
u_max = 10   # Upper bound of each element of state-feedback gain K
population_size = 200  # Increased population size
generations = 100      # Increased number of generations
cxpb = 0.8             # Crossover probability
mutpb = 0.1            # Mutation probability

# Running the solver
population, logbook = genetic_algorithm_solver(theta, M, u_min, u_max, threshold, K_LQR, population_size, generations, cxpb, mutpb)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_K = np.array(best_individual).reshape(B.shape[1], A.shape[0])
print('Best individual (K):', best_K)
print('Best fitness:', best_individual.fitness.values[0])

# Compare with LQR performance
print('LQR Gain Matrix K_LQR:', K_LQR)

# Evaluate LQR Controller
lqr_fitness = objective_function(K_LQR.flatten(), theta, M, threshold)[0]
print('LQR Controller Fitness:', lqr_fitness)

# Evaluate Best GA Controller
ga_fitness = objective_function(best_K.flatten(), theta, M, threshold)[0]
print('Best GA Controller Fitness:', ga_fitness)


In [ ]:
best_K

###  Indirect Method LMI example

In [ ]:
pip install mkl

In [ ]:
pip install cvxopt

In [ ]:
import cvxopt
#print("CVXOPT version:", cvxopt.__version__)

In [ ]:
import cvxpy as cp
import numpy as np


# System matrices (example values)
A = np.array([[0.9, 0.1],
              [0.2, 0.8]])
B = np.array([[0.1],
              [0.05]])

# Disturbance ellipsoid shape matrix Q
Q = 100*np.array([[0.01, 0],
              [0, 0.01]])

n = A.shape[0]  # State dimension
m = B.shape[1]  # Input dimension

# Variables
Y = cp.Variable((n, n), symmetric=True)
L = cp.Variable((m, n))
lambda_var = cp.Variable(nonneg=True)

# Constraints
constraints = [Y >> 1e-6 * np.eye(n), lambda_var >= 1e-6]

# RPI Condition
AY_BL = A @ Y + B @ L

# Constructing the RPI LMI
RPI_LMI = cp.bmat([
    [Y, AY_BL.T],
    [AY_BL, Y]
]) - lambda_var * cp.bmat([
    [Q, np.zeros((n, n))],
    [np.zeros((n, n)), np.zeros((n, n))]
])

constraints += [RPI_LMI >> 0]

# Objective function (e.g., minimize trace(Y) for a tighter ellipsoid)
objective = cp.Minimize(cp.trace(Y))

# Problem definition
prob = cp.Problem(objective, constraints)

# Solve the problem using CVXOPT
#prob.solve(solver=cp.CVXOPT)
prob.solve()

# Check if the problem is solved successfully
if prob.status == cp.OPTIMAL or prob.status == cp.OPTIMAL_INACCURATE:
    Y_value = Y.value
    L_value = L.value
    K = L_value @ np.linalg.inv(Y_value)
    P = np.linalg.inv(Y_value)
    print("Optimal state feedback gain K:")
    print(K)
    print("\nEllipsoidal RPI set defined by P:")
    print(P)
    eigvals = np.linalg.eigvals(P)
    print("\nEigenvalues of P:")
    print(eigvals)
else:
    print("Problem is infeasible or an error occurred.")


In [ ]:
[[255346.48788338 -22774.26218005]
 [-22774.26218005 316324.19597219]]